# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 28 2022 3:44PM,243024,12,8335473,Proven Skincare,Executing
1,Jul 28 2022 3:43PM,243023,19,ACG-2102490778,ACG North America LLC,Executing
2,Jul 28 2022 3:43PM,243023,19,ACG-2102490779,ACG North America LLC,Executing
3,Jul 28 2022 3:43PM,243023,19,ACG-2102490780,ACG North America LLC,Executing
4,Jul 28 2022 3:43PM,243023,19,ACG-2102490781,ACG North America LLC,Executing
5,Jul 28 2022 3:41PM,243019,20,ALUR687554248,Alumier Labs Inc.,Released
6,Jul 28 2022 3:41PM,243019,20,ALUR047667851,Alumier Labs Inc.,Released
7,Jul 28 2022 3:41PM,243019,20,ALUR146607454,Alumier Labs Inc.,Released
8,Jul 28 2022 3:41PM,243019,20,ALUR281120931,Alumier Labs Inc.,Released
9,Jul 28 2022 3:41PM,243019,20,ALUR512362684,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,243017,Released,1
21,243019,Released,19
22,243021,Released,1
23,243023,Executing,4
24,243024,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243017,NaN,NaN,1.0
243019,NaN,NaN,19.0
243021,NaN,NaN,1.0
243023,NaN,4.0,NaN
243024,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242901,1.0,0.0,5.0
242914,0.0,0.0,1.0
242921,36.0,13.0,8.0
242937,0.0,4.0,7.0
242964,17.0,2.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242901,1,0,5
242914,0,0,1
242921,36,13,8
242937,0,4,7
242964,17,2,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242901,1,0,5
1,242914,0,0,1
2,242921,36,13,8
3,242937,0,4,7
4,242964,17,2,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242901,1,,5
1,242914,,,1
2,242921,36,13,8
3,242937,,4,7
4,242964,17,2,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 28 2022 3:44PM,243024,12,Proven Skincare
1,Jul 28 2022 3:43PM,243023,19,ACG North America LLC
5,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.
24,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC"
25,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc."
26,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc."
27,Jul 28 2022 2:58PM,243011,22,"NBTY Global, Inc."
28,Jul 28 2022 2:40PM,243008,21,"NBTY Global, Inc."
29,Jul 28 2022 2:30PM,243006,21,"NBTY Global, Inc."
30,Jul 28 2022 2:28PM,243005,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 28 2022 3:44PM,243024,12,Proven Skincare,,1,
1,Jul 28 2022 3:43PM,243023,19,ACG North America LLC,,4,
2,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,,,19
3,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",,,1
4,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",,,1
5,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",,,1
6,Jul 28 2022 2:58PM,243011,22,"NBTY Global, Inc.",,,1
7,Jul 28 2022 2:40PM,243008,21,"NBTY Global, Inc.",,,1
8,Jul 28 2022 2:30PM,243006,21,"NBTY Global, Inc.",,,1
9,Jul 28 2022 2:28PM,243005,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 3:44PM,243024,12,Proven Skincare,,1,
1,Jul 28 2022 3:43PM,243023,19,ACG North America LLC,,4,
2,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,19,,
3,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1,,
4,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1,,
5,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",1,,
6,Jul 28 2022 2:58PM,243011,22,"NBTY Global, Inc.",1,,
7,Jul 28 2022 2:40PM,243008,21,"NBTY Global, Inc.",1,,
8,Jul 28 2022 2:30PM,243006,21,"NBTY Global, Inc.",1,,
9,Jul 28 2022 2:28PM,243005,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 3:44PM,243024,12,Proven Skincare,,1,
1,Jul 28 2022 3:43PM,243023,19,ACG North America LLC,,4,
2,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,19,,
3,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1,,
4,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 3:44PM,243024,12,Proven Skincare,NaN,1.0,NaN
1,Jul 28 2022 3:43PM,243023,19,ACG North America LLC,NaN,4.0,NaN
2,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,19.0,NaN,NaN
3,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 3:44PM,243024,12,Proven Skincare,0.0,1.0,0.0
1,Jul 28 2022 3:43PM,243023,19,ACG North America LLC,0.0,4.0,0.0
2,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,19.0,0.0,0.0
3,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,485851,8.0,4.0,0.0
12,243024,0.0,1.0,0.0
16,971956,4.0,0.0,0.0
19,485944,8.0,17.0,36.0
20,728884,39.0,2.0,18.0
21,729019,3.0,0.0,0.0
22,972019,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,485851,8.0,4.0,0.0
1,12,243024,0.0,1.0,0.0
2,16,971956,4.0,0.0,0.0
3,19,485944,8.0,17.0,36.0
4,20,728884,39.0,2.0,18.0
5,21,729019,3.0,0.0,0.0
6,22,972019,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,4.0,0.0
1,12,0.0,1.0,0.0
2,16,4.0,0.0,0.0
3,19,8.0,17.0,36.0
4,20,39.0,2.0,18.0
5,21,3.0,0.0,0.0
6,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,12,Released,0.0
2,16,Released,4.0
3,19,Released,8.0
4,20,Released,39.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,36.0,18.0,0.0,0.0
Executing,4.0,1.0,0.0,17.0,2.0,0.0,0.0
Released,8.0,0.0,4.0,8.0,39.0,3.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,0.0,0.0,0.0,36.0,18.0,0.0,0.0
1,Executing,4.0,1.0,0.0,17.0,2.0,0.0,0.0
2,Released,8.0,0.0,4.0,8.0,39.0,3.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,0.0,0.0,0.0,36.0,18.0,0.0,0.0
1,Executing,4.0,1.0,0.0,17.0,2.0,0.0,0.0
2,Released,8.0,0.0,4.0,8.0,39.0,3.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()